In [9]:
%pip check openai


No broken requirements found.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import DeepLake
import warnings
warnings.filterwarnings("ignore")

from dotenv import load_dotenv
import os

# os.environ.clear()
# RuntimeError: Could not determine home directory.
load_dotenv()
api_key = os.environ['OPENAI_API_KEY']
token = os.environ['ACTIVELOOP_TOKEN']

In [ ]:
#!git clone https://github.com/KillianLucas/open-interpreter.git ./../../

In [3]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
repo_path = "../../../open-interpreter"

# load the repository
loader = GenericLoader.from_filesystem(
    repo_path+"/interpreter",
    glob="**/*",
    suffixes=[".py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)
documents = loader.load()

len(documents)

9

In [4]:
#split the docs into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON, chunk_size=3000, chunk_overlap=50)
texts= text_splitter.split_documents(documents)

len(texts)

35

In [5]:
for  text in texts:
    print(f' {text}')

 page_content='"""\nRight off the bat, to any contributors (a message from Killian):\n\nFirst of all, THANK YOU. Open Interpreter is ALIVE, ALL OVER THE WORLD because of YOU.\n\nWhile this project is rapidly growing, I\'ve decided it\'s best for us to allow some technical debt.\n\nThe code here has duplication. It has imports in weird places. It has been spaghettified to add features more quickly.\n\nIn my opinion **this is critical** to keep up with the pace of demand for this project.\n\nAt the same time, I plan on pushing a significant re-factor of `interpreter.py` and `code_interpreter.py` ~ September 11th.\n\nAfter the re-factor, Open Interpreter\'s source code will be much simpler, and much more fun to dive into.\n\nEspecially if you have ideas and **EXCITEMENT** about the future of this project, chat with me on discord: https://discord.gg/6p3fD6rBVm\n\n- killian\n"""\n\nimport argparse\nimport os\nfrom dotenv import load_dotenv\nimport requests\nfrom packaging import version\nim

In [7]:
username="huangliang"
embeddings = OpenAIEmbeddings(disallowed_special=())
db = DeepLake(dataset_path=f"hub://{username}/interpreter", embedding=embeddings,token=token)
db.add_documents(texts)


Your Deep Lake dataset has been successfully created!


creating embeddings:   0%|          | 0/9 [00:12<?, ?it/s]


IndexError: list index out of range

In [ ]:
db = DeepLake(dataset_path='hub://huangliang/interpreter', read_only=True,embedding=embeddings,token=token)

Deep Lake Dataset in hub://huangliang/open_chat already exists, loading from the storage


In [1]:
retriever=db.as_retriever()
#distance_metric计算两个的点的距离
retriever.search_kwargs['distance_metric'] = 'cos'
#fetch_k 从文档中要检索的数据点的数据
retriever.search_kwargs['fetch_k'] = 100
retriever.search_kwargs['maximal_marginal_relevance'] = True
#k 返回相似度最高的向量的数量  
retriever.search_kwargs['k'] = 10

NameError: name 'db' is not defined

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

model = ChatOpenAI(model="gpt-3.5-turbo-16k",openai_api_key=api_key,temperature=0.1)
qa = ConversationalRetrievalChain.from_llm(model, retriever=retriever)

NameError: name 'retriever' is not defined

In [ ]:
questions = [
    """
    请为我提供有关OpenChat项目的信息。
    我已经在本地安装了该项目的代码，并希望了解如何使用OpenChat爬取网站内容并定期自动更新其知识库。
    """
]
chat_history = []

for question in questions:
    result = qa({"question":question,"chat_history":chat_history})
    chat_history.append((question,result['answer']))
    print(f"Q:\n {question} \n")
    print(f"A:\n {result['answer']} \n")

ValueError: specified dataset is empty

In [ ]:
def ask(question,chat_history):
    res = qa({"question":question,"chat_history":chat_history})
    print(f"Q:\n {question} \n")
    print(f"A:\n {res['answer']} \n")

In [ ]:
ask("What's the main programming language used in gpt-engineer?",chat_history)

Q:
 What's the main programming language used in gpt-engineer? 

A:
 The primary programming language used in GPT-Engineer is Python. 



In [ ]:
ask("What's python class to generate the prompt in gpt-engineer",chat_history)

Q:
 What's python class to generate the prompt in gpt-engineer 

A:
 The Python class used to generate prompts in GPT-Engineer is the `AI` class. It serves as the main interface to the GPT-4 model and provides methods to start a conversation with the model, continue an existing conversation, and format system and user messages. 



In [ ]:
ask("Can tell me the file in 'preprompts' floder of gpt-engineer?",chat_history)

Q:
 Can tell me the file in 'preprompts' floder of gpt-engineer? 

A:
 In the 'preprompts' folder of GPT-Engineer, you can find the following files:

1. fix_code
2. generate
3. philosophy
4. qa
5. respec
6. spec
7. unit_tests
8. use_feedback

These files contain predefined prompts that guide the AI in performing different tasks. 



In [ ]:
ask("Tell me the order of gpt-engineer use the prompt file in 'preprompts' floder?",chat_history)

Q:
 Tell me the order of gpt-engineer use the prompt file in 'preprompts' floder? 

A:
 The order of using the prompt file in the 'preprompts' folder for GPT-Engineer is as follows:

1. Fix Code (`gpt_engineer/preprompts/fix_code`)
2. Generate (`gpt_engineer/preprompts/generate`)
3. Philosophy (`gpt_engineer/preprompts/philosophy`)
4. QA (`gpt_engineer/preprompts/qa`)
5. Respec (`gpt_engineer/preprompts/respec`)
6. Spec (`gpt_engineer/preprompts/spec`)
7. Unit Tests (`gpt_engineer/preprompts/unit_tests`)
8. Use Feedback (`gpt_engineer/preprompts/use_feedback`)

This order represents a sequence of different tasks that the AI can perform based on the instructions provided in each prompt. It starts with fixing code, followed by generating code, considering best practices, seeking clarification, reviewing specifications, writing unit tests, and utilizing user feedback for code generation. 



In [ ]:
ask("Tell me the Python class of gpt-engineer use the prompt file in 'preprompts' floder?",chat_history)

Q:
 Tell me the Python class of gpt-engineer use the prompt file in 'preprompts' floder? 

A:
 The Python class used by gpt-engineer to access the prompt file in the 'preprompts' folder is the `DB` class. 



In [ ]:
ask("Tell me what dose DB.py do in gpt-engineer ?",chat_history)

Q:
 Tell me what dose DB.py do in gpt-engineer ? 

A:
 The purpose of `DB.py` in GPT-Engineer is to represent a simple database that stores its data as files in a directory. It serves as a key-value store, where keys are filenames and values are file contents. The `DB` class provides methods to check if a key (filename) exists in the database, get the value (file content) associated with a key, and set the value associated with a key. The `DB` class is responsible for managing the data used by the GPT-Engineer system. 



In [ ]:
ask("Tell me Where dose DB.py store the data in gpt-engineer ?",chat_history)

Q:
 Tell me Where dose DB.py store the data in gpt-engineer ? 

A:
 The `DB.py` file in `gpt-engineer` represents a simple database that stores its data as files in a directory. The directory where the data is stored can be specified when creating an instance of the `DB` class by providing a path argument. If the directory does not already exist, the `DB` class will create it. Each file in the directory represents a key-value pair, where the filename serves as the key and the file content serves as the value. 

